In [2]:
import pandas as pd

# Load data from CSV files
df_win_rates = pd.read_csv("C:\\Users\\eyas1\\OneDrive\\Desktop\\UWM fall 2023 Smash Data\\Cream City Clash fall 2023.csv")
df_head_to_head = pd.read_csv("C:\\Users\\eyas1\\OneDrive\\Desktop\\UWM fall 2023 Smash Data\\REAL Cream City Clash Fall 2023 H2H Data.csv")

# Reshape the head-to-head data into long format
df_head_to_head_long = pd.melt(df_head_to_head, id_vars=['Players'], var_name='Opponent', value_name='Head_to_Head_Wins')

# Merge DataFrames on the common column ('Players')
df_combined = pd.merge(df_win_rates, df_head_to_head_long, left_on='Players', right_on='Players', how='left')
# Assign weights
weight_win_rate = 0.40
weight_head_to_head = 0.60

# Calculate the weighted average separately for win rate and head-to-head
df_combined['Weighted_Win_Rate'] = weight_win_rate * df_combined['Average Win rate']
df_combined['Weighted_Head_to_Head'] = weight_head_to_head * df_combined.groupby('Players')['Head_to_Head_Wins'].transform('mean')

# Calculate the total weighted average
df_combined['Weighted Average'] = df_combined['Weighted_Win_Rate'] + df_combined['Weighted_Head_to_Head']

# Sort by the weighted average to get the top players
df_combined = df_combined.sort_values(by='Weighted Average', ascending=False)

# Extract the top 10 players
top_players = df_combined[['Players', 'Weighted Average']].drop_duplicates()

# Display or save the results
print(top_players)

# Save the top_players DataFrame to a CSV file
top_players.to_csv("top_players_performance_full_list_TEST.csv", index=False)


                  Players  Weighted Average
2                 Samurai         31.744000
38                   Eyas         30.790909
60                Sophist         30.481818
85                  Kaiju         28.074182
116                 CHICO         26.218182
139                ICHIGO         26.131636
157             TheDood22         24.713455
186               Pidgeon         23.549818
209                   Cal         23.190909
227                Shinta         21.654545
257             Frog Lady         20.518182
281             bellyache         19.406545
299          DrMichaelBay         19.118182
328                  mjmd         16.836364
356                Payday         10.195273
400        The_Verminator          9.918182
388                Thumbs          9.867273
425                 Zxtra          9.636364
448             Rainstorm          8.445455
493               TDollas          3.113818
463                goober          2.713818
520                 Helix       

In [2]:
import pandas as pd 
import plotly.express as px

file_path = 'C:\\Users\\eyas1\\Dataviz.py\\my_venv\\Cream City Clash\\top_players_performance.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

fig = px.bar(df, x= 'Players', y= 'Weighted Average')

fig.update_layout(
    title = 'Top 10 players based on wieghted average',
    title_x = 0.5, 
    xaxis_title='',
    template='plotly_dark'
)

fig.update_traces(marker_color=df['Weighted Average'])

fig.show()


In [39]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px

# Assuming df is your DataFrame with relevant player performance metrics

# Step 1: Data Preparation
features = df_win_rates[['Placement Average', 'Average Win rate','Tournaments Entered']]  # Adjust feature selection as needed

min_tournaments = 3
filtered_df = df_win_rates[df_win_rates['Tournaments Entered'] >= min_tournaments]
filtered_features = filtered_df[['Placement Average', 'Average Win rate', 'Tournaments Entered']]

# Step 2: Normalization
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Step 3: Convert normalized features back to a DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=features.columns)

# Step 4: Choose the Number of Clusters (K)
# You may use the elbow method or other techniques to find the optimal K
k = 5

# Step 5: Clustering
kmeans = KMeans(n_clusters=k, random_state=42)
df_win_rates['Cluster'] = kmeans.fit_predict(scaled_features)

# Visualize the clusters using Plotly Express
fig = px.scatter(filtered_df, x='Placement Average', y='Average Win rate', color='Cluster', title='Player Clusters',
                 labels={'Placement Average': 'Placement Average', 'Average Win rate': 'Average Win Rate'},
                 template='plotly_dark')

fig.show()



c:\Users\eyas1\Conda_time\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\eyas1\Conda_time\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [5]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('C:\\Users\\eyas1\\OneDrive\\Desktop\\UWM fall 2023 Smash Data\\Cream City Clash fall 2023.csv')

# Assuming 'tournaments entered' and 'sets played' are the column names
# Create a new column for the ratio of sets played to tournaments entered
df['Sets per Tournament'] = df['Total Sets Played'] / df['Tournaments Entered']

# Display or save the updated DataFrame
print(df)

# Save the updated DataFrame to a new CSV file
df.to_csv("sets_per_tournament.csv", index=False)



                 Players  Tournaments Entered  Total Sets Played  Sets won  \
0         Baskin Robbins                    1                  8         8   
1                  comet                    1                  7         6   
2                Samurai                    9                 72        56   
3                Spencer                    1                  8         6   
4                   Eyas                    8                 65        49   
5                Sophist                   10                 72        55   
6                 Toland                    1                  7         5   
7                  Kaiju                    9                 69        48   
8             Dinkleberg                    1                  3         2   
9                DoubleW                    2                 12         8   
10                 CHICO                    5                 33        22   
11                ICHIGO                   12                 83

In [26]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the CSV files
df_weighted_average = pd.read_csv("top_players_performance_full_list.csv")
df_sets_per_tournament = pd.read_csv("sets_per_tournament.csv")

# Merge the DataFrames on a common column, assuming 'Player' is the common column
df_combined = pd.merge(df_weighted_average, df_sets_per_tournament, on='Players', how='inner')

# Drop rows with missing values
df_combined = df_combined.dropna()

# Select the columns for clustering
data_for_clustering = df_combined[['Weighted Average', 'Sets per Tournament', 'Players']]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_clustering[['Weighted Average', 'Sets per Tournament']])

# Apply K-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)  # You can choose the number of clusters
df_combined['Cluster'] = kmeans.fit_predict(scaled_data)

# Create a 3D scatter plot using Plotly Express with dark theme
fig = px.scatter_3d(df_combined, x='Weighted Average', y='Sets per Tournament', z='Cluster',
                    color='Cluster', hover_data=['Players'],
                    title='3D K-Means Clustering',
                    labels={'Weighted Average': 'Weighted Average', 'Sets per Tournament': 'Sets per Tournament'},
                    template='plotly_dark')  # Apply dark theme

# Show the plot
fig.show()


c:\Users\eyas1\Conda_time\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\eyas1\Conda_time\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [37]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the CSV files
df_weighted_average = pd.read_csv("top_players_performance_full_list.csv")
df_sets_per_tournament = pd.read_csv("sets_per_tournament.csv")

# Merge the DataFrames on a common column, assuming 'Player' is the common column
df_combined = pd.merge(df_weighted_average, df_sets_per_tournament, on='Players', how='inner')

# Drop rows with missing values
df_combined = df_combined.dropna()

# Change the scale of 'Sets per Tournament' to go up by 0.5
df_combined['Sets per Tournament'] = df_combined['Sets per Tournament'] * 0.5

# Select the columns for clustering
data_for_clustering = df_combined[['Weighted Average', 'Sets per Tournament', 'Players']]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_clustering[['Weighted Average', 'Sets per Tournament']])

# Apply K-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)  # You can choose the number of clusters
df_combined['Cluster'] = kmeans.fit_predict(scaled_data)

# Invert the cluster scale
df_combined['Cluster'] = 3 - df_combined['Cluster']

# Create a 3D scatter plot using Plotly Express with dark theme
fig = px.scatter_3d(df_combined, x='Weighted Average', y='Sets per Tournament', z='Cluster',
                    color='Cluster', hover_data=['Players'],
                    title='3D K-Means Clustering',
                    labels={'Weighted Average': 'Weighted Average', 'Sets per Tournament': 'Sets per Tournament'},
                    template='plotly_dark')  # Apply dark theme

# Show the plot
fig.show()


c:\Users\eyas1\Conda_time\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\eyas1\Conda_time\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [22]:

import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the CSV files
df_weighted_average = pd.read_csv("C:\\Users\\eyas1\\Dataviz.py\\my_venv\\Cream City Clash\\top_players_performance_full_list.csv")
df_sets_per_tournament = pd.read_csv("sets_per_tournament.csv")

# Merge the DataFrames on a common column, assuming 'Player' is the common column
df_combined = pd.merge(df_weighted_average, df_sets_per_tournament, on='Players', how='inner')

# Drop rows with missing values
df_combined = df_combined.dropna()

# Change the scale of 'Sets per Tournament' to go up by 0.5
df_combined['Sets per Tournament'] = df_combined['Sets per Tournament'] 

# Select the columns for clustering
data_for_clustering = df_combined[['Players', 'Weighted Average', 'Sets per Tournament']]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_clustering[['Weighted Average', 'Sets per Tournament']])

# Apply K-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)  # You can choose the number of clusters
df_combined['Cluster'] = kmeans.fit_predict(scaled_data)

# Invert the cluster scale
df_combined['Cluster'] = 3 - df_combined['Cluster']

# Create a 3D scatter plot using Plotly Express with dark theme
fig = px.scatter_3d(df_combined, x='Players', y='Sets per Tournament', z='Weighted Average',
                    color='Cluster', hover_data=['Players'],
                    title='3D K-Means Clustering',
                    labels={'Weighted Average': 'Weighted Average', 'Sets per Tournament': 'Sets per Tournament'},
                    category_orders={'Players': df_combined['Players']},  # Specify the order of players
                    template='plotly_dark') # Apply dark theme

# Manually set the y-axis range
max_weighted_average = df_combined['Weighted Average'].max()
buffer_space = 1.0  # You can adjust this value based on your preference
fig.update_yaxes(range=[5, max_weighted_average + buffer_space])

# Update layout to hide tick labels on the x-axis
fig.update_layout(scene=dict(xaxis=dict(showticklabels=False)))

# Update layout to remove the color bar
fig.update_layout(coloraxis_showscale=False)

# Show the plot
fig.show()




c:\Users\eyas1\Conda_time\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

c:\Users\eyas1\Conda_time\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

